## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 4.6MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=d581daa8c55cdacb52b768f39886d3a6edf01eb9797f91dbdd3f1517091d3c9f
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=02c9c759254302d6313cd545c47c99da898a0b28edf1325ced5bd34c724b5671
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.3.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
#scaling
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.4937 - accuracy: 0.8299 - val_loss: 0.4011 - val_accuracy: 0.8507
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3575 - accuracy: 0.8660 - val_loss: 0.3624 - val_accuracy: 0.8635
Epoch 3/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.3258 - accuracy: 0.8789 - val_loss: 0.3554 - val_accuracy: 0.8718


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4878 - accuracy: 0.8328 - val_loss: 0.4110 - val_accuracy: 0.8533
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3534 - accuracy: 0.8706 - val_loss: 0.3738 - val_accuracy: 0.8638
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3192 - accuracy: 0.8822 - val_loss: 0.3623 - val_accuracy: 0.8677


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4509 - accuracy: 0.8411 - val_loss: 0.3555 - val_accuracy: 0.8722
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3282 - accuracy: 0.8786 - val_loss: 0.3346 - val_accuracy: 0.8772
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2907 - accuracy: 0.8927 - val_loss: 0.3341 - val_accuracy: 0.8797


Epoch 1/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3947 - accuracy: 0.8585 - val_loss: 0.3055 - val_accuracy: 0.8917
Epoch 2/3
1688/1688 [==============================] - 6s 3ms/step - loss: 0.2503 - accuracy: 0.9080 - val_loss: 0.2545 - val_accuracy: 0.9103
Epoch 3/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1898 - accuracy: 0.9295 - val_loss: 0.2524 - val_accuracy: 0.9105


Epoch 1/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3902 - accuracy: 0.8600 - val_loss: 0.2914 - val_accuracy: 0.8928
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2523 - accuracy: 0.9067 - val_loss: 0.2524 - val_accuracy: 0.9057
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1935 - accuracy: 0.9289 - val_loss: 0.2410 - val_accuracy: 0.9127


INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        153648    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2230368   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,386,266
Trainable params: 2,386,266
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1467 - accuracy: 0.9461 - val_loss: 0.2477 - val_accuracy: 0.9128
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1107 - accuracy: 0.9586 - val_loss: 0.2856 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0798 - accuracy: 0.9703 - val_loss: 0.3200 - val_accuracy: 0.9145
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0576 - accuracy: 0.9784 - val_loss: 0.3368 - val_accuracy: 0.9147
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0451 - accuracy: 0.9838 - val_loss: 0.3739 - val_accuracy: 0.9090
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0340 - accuracy: 0.9873 - val_loss: 0.4241 - val_accuracy: 0.9112
Epoch 10/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0313 - accuracy: 0.9887 - val_loss: 0.4644 - val_accuracy

In [21]:
pred=model.predict(test_images)

In [24]:
predicted_class=pred.argmax(axis=1)

In [20]:
test_labels

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [30]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [28]:
confusion_matrix(test_labels,predicted_class)

array([[844,   2,  25,  14,   4,   1, 103,   0,   7,   0],
       [  2, 984,   2,   8,   0,   0,   2,   0,   2,   0],
       [ 11,   0, 921,   9,  19,   0,  39,   0,   1,   0],
       [ 16,   8,  20, 910,  12,   0,  32,   0,   2,   0],
       [  3,   1, 116,  33, 777,   0,  69,   0,   1,   0],
       [  0,   0,   1,   0,   0, 978,   0,  15,   0,   6],
       [ 83,   2,  98,  21,  37,   0, 751,   0,   8,   0],
       [  0,   0,   0,   0,   0,   6,   0, 974,   0,  20],
       [  1,   0,   5,   1,   3,   1,   4,   4, 981,   0],
       [  1,   0,   0,   0,   0,   4,   0,  28,   0, 967]])

In [31]:
accuracy_score(test_labels,predicted_class)

0.9087